In [1]:
import os
from pickle import load
from argparse import ArgumentParser
from glob import glob

import pandas as pd
import numpy as np
import random

from math import exp, log, floor, ceil
from igraph import Graph

import numpy as np
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data.sampler import Sampler
import sklearn
import itertools
import logging
import igraph
from sklearn import preprocessing


NUM_ROW_DISEASE = 300000000
NUM_ROW_POP = 100000
PERCENT = 0.01

/home/ljxw88/anaconda3/envs/hgcn/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DELTA = 7 # how far a period to look ahead for
PAST_WINDOW = 3 # how far back to look for infection status

def write_vertex_features(vertex_features, subgraph, age_map):
    # for each node in subgraph, write features to vertex_features
    # age
    # coreness
    # authority
    # rareness
    subgraph_pids = subgraph.vs["name"]
    
    coreness = subgraph.coreness()
    authority = subgraph.authority_score(weights="duration")
    degrees = np.array([subgraph.degree(i) for i in range(len(subgraph_pids))])

    zero_degrees = degrees == 0
    degrees[zero_degrees] = 1
    rareness = 1.0/degrees
    rareness[zero_degrees] = 0
    
    vertex_features[tuple(subgraph_pids), 0] = [age_map[pid] for pid in subgraph_pids]
    vertex_features[tuple(subgraph_pids), 1] = [coreness[subgraph.vs.find(name=pid).index] for pid in subgraph_pids]
    vertex_features[tuple(subgraph_pids), 2] = [authority[subgraph.vs.find(name=pid).index] for pid in subgraph_pids]

    vertex_features[tuple(subgraph_pids), 3] = rareness
    
    return vertex_features

def process_subgraph(subgraph, si_table, age_map, vertex_features, ego_pid, cutoff=None, max_size=50):

    # for each infected node in subgraph create
    # a positive instance, at time t-delta

    # create a negative instance for neighbors not infected at time - delta

    subgraph_pids = subgraph.vs["name"]
    si_subgraph = si_table[si_table.index.get_level_values("pid").isin(subgraph_pids)]

    if len(si_subgraph) == 0:
        return None

    pid_arr = np.pad(np.array(subgraph_pids), (0, max_size-len(subgraph_pids)), "constant", constant_values=0)
    valid_days = si_subgraph.index.get_level_values("infected")
    if cutoff:
        valid_days = valid_days[valid_days >= cutoff]

    n = len(valid_days.unique())

    labels = np.zeros((n,), dtype=int)
    influence_feature = np.zeros((n,max_size,2))
    adjacency_matrix = np.zeros((n,max_size, max_size))
    vertex_id = np.zeros((n,max_size), dtype=int)

    subgraph_adj = np.array(subgraph.get_adjacency().data)
    pad_num = max_size - subgraph_adj.shape[0]  
    adj = np.pad(subgraph_adj, (0,pad_num), "constant", constant_values=0)
    
    vertex_features = write_vertex_features(vertex_features, subgraph, age_map)
    
    label_idx = 0

    for day in valid_days.unique():
        # look forward delta steps to identify 
        min_day = day - PAST_WINDOW

        day_level = si_subgraph.index.get_level_values("infected").to_series()
        back_subgraph = si_subgraph[day_level.between(min_day, day).values]
    
        state_t_d = si_subgraph[day_level.between(day, day+DELTA, inclusive="right").values]

        labels[label_idx] = ego_pid in state_t_d.index.get_level_values("pid")
        adjacency_matrix[label_idx] = adj
        vertex_id[label_idx] = pid_arr

        influence_feature[label_idx,:,0] = (pid_arr == ego_pid).astype(int)

        # get neighbors who are infected at time t-delta 
        is_inf = np.isin(pid_arr, back_subgraph.index.get_level_values("pid").unique()).astype(int)
        influence_feature[label_idx,:,1] = is_inf
       
        label_idx += 1
 
    return (labels, adjacency_matrix, influence_feature, vertex_id, vertex_features)

def rand_walk_igraph(graph, start_node, size, restart_prob, wname="weight"):
    """
    Do an edge-weighted random walk with restart starting from start_node
    on graph.

    graph - an igraph graph with vertex attributes "name" and edge attributes wname
    start_node - a vertex in graph (note that this *should not* be the vertex name)
    size - the maximum size of the random walk
    restart_prob - the probability of returning to start_node at each step
    wname (optional) - the name of the edge attribute to weight

    returns - subgraph (an igraph graph produced through the random walk)
    """
    nodes = set([start_node])
    current = start_node

    # To avoid a situation where we end up in a loop, we limit the number
    # of steps we can take without adding a new node

    max_step = 200
    step = 0

    while len(nodes) < size:
        curr_size = len(nodes) # TODO: preprocess pid_part to ensure degree > 0 => avoid max_step iterations
        if random.random() < restart_prob or len(graph.neighbors(current)) == 0:
            current = start_node
        else:
            poss_edges = graph.incident(current)
            poss_weights = np.array([graph.es[n][wname] for n in poss_edges])
            if len(poss_edges) == 0:
                None
                # print(f"There are no incident edges for {current}, but neighbors {graph.neighbors(current)}")
            if poss_weights.sum() == 0:
                # print(f"Edge weights are 0 for {current}")
                current = start_node
                # TODO: make into exception
            else:
                #poss_weights = np.nan_to_num(poss_weights)
                new_edge = np.random.choice(poss_edges, p = poss_weights/poss_weights.sum())

                if graph.es[new_edge].source == current:
                    current = graph.es[new_edge].target
                else:
                    current = graph.es[new_edge].source 
            nodes.add(current)
        if len(nodes) == curr_size:
            step += 1
        else:
            step = 0
        if step == max_step:
            return graph.induced_subgraph(list(nodes))
    return graph.induced_subgraph(list(nodes))

In [3]:
disease_outcome_data_path = './data/pandemic/centralized/train/va_disease_outcome_training.csv.gz'
person_data_path = './data/pandemic/centralized/train/va_person.csv.gz'
population_network_data_path = './data/pandemic/centralized/train/va_population_network.csv.gz'

disease_outcome_df = pd.read_csv(disease_outcome_data_path, compression='gzip', nrows=NUM_ROW_DISEASE)
person_df = pd.read_csv(person_data_path, compression='gzip')
population_network_df = pd.read_csv(population_network_data_path, compression='gzip', nrows=NUM_ROW_POP)

: 

: 

In [ ]:
# read in population file (person_df)
print("Size of person_df: {}".format(person_df.shape[0]))
pop_file = person_df
age_map = pop_file[["pid", "age"]].set_index("pid")["age"].to_dict()
max_id = max(age_map.keys())
print(f"Made age map, {max_id} is the max_id")
vertex_features = np.zeros((max_id+1, 4))

Size of person_df: 7688059
Made age map, 7688058 is the max_id


In [ ]:
# read in subgraph list (disease_outcome_df)
disease_data = disease_outcome_df
is_infected = disease_data["state"] == "I"
pid = disease_data[is_infected]["pid"]
day = disease_data[is_infected]["day"]

inf_time_df = pd.DataFrame({"pid" : pid, "day" : day})

is_rec = disease_data["state"] == "R"
pid = disease_data[is_rec]["pid"]
day = disease_data[is_rec]["day"]

rec_time_df = pd.DataFrame({"pid" : pid, "day" : day})

def lookup_rec(row):
    pid_subset = rec_time_df[(rec_time_df["pid"] == row["pid"]) & (rec_time_df["day"] >= row["day"])]
    if pid_subset.shape[0] > 0:
        return pid_subset["day"].min()
    else:
        return 0

recovery_times = inf_time_df.apply(lookup_rec, axis=1)
si_table = inf_time_df
si_table.rename({"day" : "infected"}, axis=1, inplace=True)

si_table["recovery"] = recovery_times

si_table.set_index(["pid", "infected"], verify_integrity=True, inplace=True)
si = si_table
print(f"Total si: {si.shape[0]}")

Total si: 38040


In [ ]:
# (population_network_df)
edges = population_network_df
edges["pid1s"] = edges.apply(lambda row: min(row["pid1"], row["pid2"]), axis=1)
edges["pid2s"] = edges.apply(lambda row: max(row["pid1"], row["pid2"]), axis=1)

edges = edges.drop(columns=['pid1', 'pid2'])
edges = edges.rename(columns={'pid1s':'pid1', 'pid2s':'pid2'})

collapsed_edges = edges[["pid1", "pid2", "duration"]].groupby(["pid1", "pid2"]).sum()

graph_df = collapsed_edges.reset_index()
graph_df.head()
tuples = [tuple(x) for x in graph_df.values]
graph = Graph.TupleList(tuples, directed = True, edge_attrs = ['duration'])
vertex_names = list(graph.vs["name"])
vertex_names.sort()

print(f"There are {len(vertex_names)} vertices")
graph_dict = {}
n_dict = 0

pids = vertex_names
node = []
for pid in pids:
    node = graph.vs.find(name=pid) # name
    subgraph = rand_walk_igraph(graph, node, 50, 0.8, wname="duration")
    graph_dict[pid] = subgraph

subgraphs = graph_dict
print(f"There are {len(subgraphs)} subgraphs")
sample_size = 300000
index_set = np.random.choice(list(subgraphs.keys()), replace=False, size=min(sample_size, len(subgraphs)))

There are 35113 vertices
There are 35113 subgraphs


In [ ]:
label_list = []
adj_list = []
inf_list = []
vert_id = []
for i,subgraph_pid in enumerate(index_set): 

    subgraph = subgraphs[subgraph_pid]
    output = process_subgraph(subgraph, si, age_map, vertex_features, subgraph_pid, None)

    if output is not None:
        labels, adjacency_matrix, influence_feature, vertex_id, vertex_features = output

        label_list.append(labels)
        adj_list.append(adjacency_matrix)
        inf_list.append(influence_feature)
        vert_id.append(vertex_id)

    # if (i+1) % 100 == 0:
    #     print(f"Handled {i}/{len(index_set)} instances, {len(label_list)} added")
print(f"Total subgraphs: {len(label_list)}")

Total subgraphs: 237


In [ ]:
label_list_np = np.concatenate(label_list, dtype=int)
vert_id_np = np.concatenate(vert_id)
adj_list_np = np.concatenate(adj_list)
inf_list_np = np.concatenate(inf_list)
vertex_features_np = vertex_features

In [ ]:
# compose edgelist
def gen_edges(matrix, vertex_ids):

    edges = matrix.nonzero()

    for e1, e2 in zip(edges[0], edges[1]):
        yield vertex_ids[e1], vertex_ids[e2]

def gen_graph(adj_mat, vertex_ids):

    n = adj_mat.shape[0]

    for i in range(n):
        for e1, e2 in gen_edges(adj_mat[i], vertex_ids[i]):
            yield e1, e2

adj_mat = adj_list_np
print(f"Translating {adj_mat.shape[0]} adjacency matrices to edgelist")
vertex_ids = vert_id_np
assert adj_mat.shape[0] == vertex_ids.shape[0]
for e1, e2 in gen_graph(adj_mat, vertex_ids):
    print(e1, e2)

Translating 756 adjacency matrices to edgelist
368061 1891118
368061 4440707
368061 1944263
368061 3616227
368061 4090941
368061 5130445
368061 5916864
368061 5917584
368061 5918250
368061 5918944
368061 5921963
368061 5931659
368061 5932163
368061 5935467
368061 6278624
1891118 5935467
1891118 4413989
4440707 5917584
4440707 5935467
3616227 6278624
4090941 5130445
4090941 5916864
5130445 5917584
5130445 5918250
5130445 5932163
5130445 5933269
5916864 5918944
5916864 6278624
5917584 5935467
5931659 6278624
5931659 5933269
5935467 6278624
4413989 4440707
4413989 5935467
4413989 6278624
5933269 6278624
368061 1891118
368061 4440707
368061 1944263
368061 3616227
368061 4090941
368061 5130445
368061 5916864
368061 5917584
368061 5918250
368061 5918944
368061 5921963
368061 5931659
368061 5932163
368061 5935467
368061 6278624
1891118 5935467
1891118 4413989
4440707 5917584
4440707 5935467
3616227 6278624
4090941 5130445
4090941 5916864
5130445 5917584
5130445 5918250
5130445 5932163
5130445

In [ ]:
labels = label_list_np
adj = adj_list_np
vertex_id = vert_id_np
inf_feature = inf_list_np
vertex_feat = vertex_features_np
print(f"There are {labels.sum()} positive instances to {len(labels)} total instances")

There are 126 positive instances to 756 total instances


In [ ]:
class ChunkSampler(Sampler):
    """
    Samples elements sequentially from some offset.
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(int(self.start), int(self.start) + int(self.num_samples)))

    def __len__(self):
        return self.num_samples


class InfluenceDataSet(Dataset):
    def __init__(self, file_dir, embedding_dim, seed, shuffle, model, max_vertex_idx=None):
        self.graphs = np.load(os.path.join(file_dir, "adjacency_matrix.npy")).astype(np.float32)

        # self-loop trick, the input graphs should have no self-loop
        identity = np.identity(self.graphs.shape[1])
        self.graphs += identity
        self.graphs[self.graphs != 0] = 1.0
        if model == "gat" or model == "pscn":
            self.graphs = self.graphs.astype(np.dtype('B'))
        elif model == "gcn":
            # normalized graph laplacian for GCN: D^{-1/2}AD^{-1/2}
            for i in range(len(self.graphs)):
                graph = self.graphs[i]
                d_root_inv = 1. / np.sqrt(np.sum(graph, axis=1))
                graph = (graph.T * d_root_inv).T * d_root_inv
                self.graphs[i] = graph
        else:
            raise NotImplementedError
        print("graphs loaded!")

        # wheather a user has been influenced
        # wheather he/she is the ego user
        self.influence_features = np.load(
                os.path.join(file_dir, "influence_feature.npy")).astype(np.float32)
        print("influence features loaded!")

        self.labels = np.load(os.path.join(file_dir, "label.npy"))
        print("labels loaded!")

        self.vertices = np.load(os.path.join(file_dir, "vertex_id.npy"))
        print("vertex ids loaded!")

        if shuffle:
            self.graphs, self.influence_features, self.labels, self.vertices = \
                    sklearn.utils.shuffle(
                        self.graphs, self.influence_features,
                        self.labels, self.vertices,
                        random_state=seed
                    )

        vertex_features = np.load(os.path.join(file_dir, "vertex_feature.npy"))
        vertex_features = preprocessing.scale(vertex_features)
        self.vertex_features = torch.FloatTensor(vertex_features)
        print("global vertex features loaded!")

        embedding_path = os.path.join(file_dir, "deepwalk.emb_%d" % embedding_dim)
        # sometimes train and eval have different max vertex
        # but max size should be same regardless
        if max_vertex_idx is None:
            max_vertex_idx = np.max(self.vertices)
        embedding = load_w2v_feature(embedding_path, max_vertex_idx)
        self.embedding = torch.FloatTensor(embedding)
        print("%d-dim embedding loaded!", embedding_dim)

        self.N = self.graphs.shape[0]
        print("%d ego networks loaded, each with size %d" % (self.N, self.graphs.shape[1]))

        n_classes = self.get_num_class()
        class_weight = self.N / (n_classes * np.bincount(self.labels))
        self.class_weight = torch.FloatTensor(class_weight)

    def get_labels(self):
        return self.labels

    def get_embedding(self):
        return self.embedding

    def get_vertex_features(self):
        return self.vertex_features

    def get_feature_dimension(self):
        return self.influence_features.shape[-1]

    def get_num_class(self):
        return np.unique(self.labels).shape[0]

    def get_class_weight(self):
        return self.class_weight

    def __len__(self):
        return self.N

    def __getitem__(self, idx):
        return self.graphs[idx], self.influence_features[idx], self.labels[idx], self.vertices[idx]